# ETL: OFERTA AUTOBUSES ENTRE LOS AÑOS 2019 - 2021

In [1]:
import pandas as pd
import numpy as np
import glob

# Esta librería puede fallar si se ejecuta en Google Colabs y similares.
import locale
locale.setlocale(locale.LC_ALL, "es_ES")

'es_ES'

## Carga de los ficheros

In [ ]:
files = glob.glob("../datos/Coches_Cuadro_Oferta_Real/*/Coches_Cuadro_Oferta_Real_*.csv")

# Read column names from file
cols = list(pd.read_csv(files[0], sep=';',nrows=1))
cols = list(filter(lambda col: col.strip(), cols))

#Cargamos los datos y especificamos los tipos en las columnas, así como eliminar la última columna por ser vacia
dfs = [pd.read_csv(f, header=0, sep=";",encoding = "ISO-8859-1",dtype={'Elinea':'str'},usecols=cols) for f in files]
df = pd.concat(dfs,ignore_index=True)

In [ ]:
# Eliminar los datos en memeria de  los ficheros
del dfs

# Exportamos el dataframe resultante (Datos brutos, ficheros unificados).
df.to_csv('../datos/Coches_Cuadro_Oferta_Real/autobuses_oferta_real.csv')

## Creación de nuevas columnas e información

In [2]:
oferta = pd.read_csv('../datos/Coches_Cuadro_Oferta_Real/autobuses_oferta_real.csv',dtype={'Elinea':'str'})
oferta.drop(columns='Unnamed: 0',inplace=True)
oferta.shape

(3866436, 7)

1. Columna `FServicio` en formato fecha.
2. Nueva columna para saber si esa fecha era un día de lunes a viernes (True) o sábado y domingo (False)
3. Completamos la columna `Coches` con 0 si alguna fila no tiene valor.

In [3]:
# oferta['FServicio'] = pd.to_datetime(oferta['FServicio'], format='%d/%m/%Y')
# oferta.loc[oferta['FServicio'].dt.dayofweek > 4,'Diario'] = False
# oferta['Diario'].fillna(True,inplace=True)
# oferta['Coches'].fillna(0,inplace=True)

Se opta por la creación de una tabla auxiliar ya que reduce el tiempo de ejecución.
Con la tabla auxiliar es necesario solo realizar un join entre índices, reduciendo así el computo.

La otra forma era realizando la siguiente operación:

```python
%%time
oferta['fechaCorta'] = oferta['FServicio'].dt.strftime('%B.%Y')
```
| |Tiempo|
|-|-|
|CPU times: total:| 33.6 s|
|Wall time: |38.8 s|

In [4]:
%%time
tiempos = pd.DataFrame({'FServicio': pd.date_range('2019-01-01', '2021-12-31', freq='D')})
tiempos['fechaCorta'] = tiempos['FServicio'].dt.strftime('%B.%Y')
if 'fechaCorta' not in oferta.columns:
    oferta = oferta.join(tiempos.set_index('FServicio'),on='FServicio')

CPU times: total: 844 ms
Wall time: 1.2 s


In [ ]:
oferta.to_csv('../datos/output/oferta.csv',encoding='utf8',index=False)

### Calendar Dates GFTS
Este archivo contiene fechas en los que el servicio ha cambiado `Laborable` a `Festivo` excepcionalmente.

In [ ]:
calendar_dates = pd.read_csv('../datos/GFTS/gfts_calendar_dates.csv')
calendar_dates['date_formated'] = pd.to_datetime(calendar_dates['date_formated'])
calendar_festivo = calendar_dates.loc[calendar_dates['exception_type']==1,['exception_type','date_formated']].copy()
calendar_festivo = calendar_festivo[calendar_festivo['date_formated'].dt.year < 2022].drop_duplicates()
calendar_festivo.set_index(['date_formated'],inplace=True)

In [5]:
tipo_dia = pd.read_csv('../datos/tipo_dia.csv')
tipo_dia['date'] = pd.to_datetime(tipo_dia['date'],dayfirst=True)
tipo_dia['day_name'] = tipo_dia['date'].dt.day_name()
tipo_dia = tipo_dia.set_index(['date'])

In [ ]:
festivos = tipo_dia[(tipo_dia['day_name']!='Sunday') & (tipo_dia['dayType']=='FE')]

### Diferencia de días entre el GFTS y el extraido por API
``` python
calendar_festivo.index.difference(festivos.index)
```

¿Por qué al hacer merge pasamos de tener 3.5M de filas a tener 7.5M?

[Explicación de StackOverflow](https://stackoverflow.com/a/39019766)

In [ ]:
# Unimos los dataframe de oferta y calendar_festivo para cambiar a Diario=False
# las fechas que aparezcan en este segundo dataframe.

# oferta = oferta.merge(right=calendar_festivo,\
#                 right_on='date_formated',\
#                 left_on='FServicio',\
#                 how='left'\
#             ).drop(columns='date_formated')

# oferta = oferta.join(other=calendar_festivo.set_index('date_formated'), on='FServicio')
# oferta['Diario'] = oferta['exception_type']!=1
# oferta.drop_duplicates(inplace=True)
# oferta.drop(columns=['exception_type'],inplace=True)

In [ ]:
oferta = oferta.join(tipo_dia,on='FServicio')

# https://stackoverflow.com/questions/38869778/pandas-set-column-equal-to-grouped-sum-of-another-column?noredirect=1&lq=1
oferta['MediaCochesMes'] = oferta.groupby(by=['CLinea', 'fechaCorta', 'IDFranja']).Coches.transform('mean')

oferta['MediaCochesMes_Laborables'] = oferta[oferta['dayType']=='LA'].groupby(by=['CLinea','IDFranja','fechaCorta']).Coches.transform('mean')

In [12]:
oferta

,CLinea,Elinea,Denominacion,FServicio,IDFranja,Intervalo,Coches,Diario,fechaCorta,strike,dayType,day_name,MediaCochesMes,MediaCochesMes_Laborables
0,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H07,070000 - 075959,2.0,True,enero.2019,N,FE,Tuesday,5.580645,NaN
1,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H08,080000 - 085959,3.0,True,enero.2019,N,FE,Tuesday,6.580645,NaN
2,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H09,090000 - 095959,3.0,True,enero.2019,N,FE,Tuesday,7.387097,NaN
3,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H10,100000 - 105959,4.0,True,enero.2019,N,FE,Tuesday,8.903226,NaN
4,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H11,110000 - 115959,4.0,True,enero.2019,N,FE,Tuesday,9.193548,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3866431,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-29,H20,200000 - 205959,5.0,False,agosto.2021,N,FE,Sunday,7.758621,NaN
3866432,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-29,H21,210000 - 215959,5.0,False,agosto.2021,N,FE,Sunday,7.068966,NaN
3866433,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-29,H22,220000 - 225959,5.0,False,agosto.2021,N,FE,Sunday,6.379310,NaN
3866434,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-29,H23,230000 - 235959,5.0,False,agosto.2021,N,FE,Sunday,5.000000,NaN


In [15]:
# ¿Sería mejor idea separar en una nueva tabla las medidas de Mes?

oferta[oferta['MediaCochesMes_Laborables'].notna()]

,CLinea,Elinea,Denominacion,FServicio,IDFranja,Intervalo,Coches,Diario,fechaCorta,strike,dayType,day_name,MediaCochesMes,MediaCochesMes_Laborables
18,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-02,H06,060000 - 065959,4.0,True,enero.2019,N,LA,Wednesday,3.840000,4.000000
19,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-02,H07,070000 - 075959,6.0,True,enero.2019,N,LA,Wednesday,5.580645,6.857143
20,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-02,H08,080000 - 085959,7.0,True,enero.2019,N,LA,Wednesday,6.580645,7.857143
21,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-02,H09,090000 - 095959,8.0,True,enero.2019,N,LA,Wednesday,7.387097,8.857143
22,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-02,H10,100000 - 105959,10.0,True,enero.2019,N,LA,Wednesday,8.903226,10.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3866391,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-27,H20,200000 - 205959,9.0,True,agosto.2021,N,LA,Friday,7.758621,9.000000
3866392,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-27,H21,210000 - 215959,8.0,True,agosto.2021,N,LA,Friday,7.068966,8.000000
3866393,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-27,H22,220000 - 225959,7.0,True,agosto.2021,N,LA,Friday,6.379310,7.000000
3866394,781,SE,ATOCHA - NUEVOS MINISTERIOS,2021-08-27,H23,230000 - 235959,5.0,True,agosto.2021,N,LA,Friday,5.000000,5.000000


## Hora punta Madrid
Según el informe anual de [2018 de CRTM](https://www.crtm.es/media/712934/edm18_sintesis.pdf#page=54) y la [Encuesta Sintética de Movilidad - ESM14](https://www.crtm.es/media/519661/esm_2014.pdf) las franjas horarias quedan de la siguiente manera:
| Afluencia  | Intervalo horas | Tipo       | Información                                                    |
| ---------  | --------------- | ---------- | -------------------------------------------------------------- |
| Máxima     | 07:00 - 09:00   | Hora punta | Movilidad ocupacional, trabajo y estudios.                     |
| Baja       | 10:00 - 13:00   | Hora valle |                                                                |
| Media      | 13:00 - 15:00   | Hora punta | Movilidad ocupacional, trabajo y estudios.                     |
| Baja       | 15:30 - 16:00   | Hora valle |                                                                |
| Media      | 16:00 - 18:00   | Hora punta | Salida de los centros escolares o el fin de la jornada laboral.|
| Media-Baja | 18:00 - 23:00   | Decreciente| Salida de los centros escolares o el fin de la jornada laboral.|


_Los viajes por movilidad ocupacional, trabajo y estudios, se producen en dos periodos concretos, entre las 6h y las 9h de la mañana y entre la 13h y las 15 h de la tarde. Existe otro pico entre las 16h y las 18h relacionado con la salida de los centros escolares o el fin de la jornada laboral._


![Distribución horaria de los viajeros según su actividad](../markdown_images/distribución_horaria_viajeros_actividad.png)

In [13]:
HORAS = ['H07','H08','H09']

In [20]:
oferta['MediaCochesDía_HoraPunta'] = oferta[(oferta['IDFranja'].isin(HORAS)) & (oferta['dayType']=="LA")].groupby(by=['CLinea', 'FServicio'],as_index=False)['Coches'].transform('mean')

In [21]:
oferta['MediaCochesMes_HoraPunta'] = oferta[(oferta['IDFranja'].isin(HORAS)) & (oferta['dayType']=="LA")].groupby(by=['CLinea', 'fechaCorta'],as_index=False)['Coches'].transform('mean')

In [ ]:
# horapunta['MediaCochesMes_HoraPuntaLaborables'] = horapunta[horapunta['Diario']==True].groupby(by=['CLinea', 'fechaCorta'],as_index=False)['Coches'].transform('mean')

In [36]:
tmp = oferta[['CLinea','Elinea','Denominacion','fechaCorta','FServicio','IDFranja','Intervalo','dayType','day_name','Coches','MediaCochesMes','MediaCochesMes_Laborables','MediaCochesDía_HoraPunta','MediaCochesMes_HoraPunta']].copy()

In [64]:
horapunta = oferta[oferta['IDFranja'].isin(HORAS)].copy()

In [80]:
oferta = pd.concat([oferta,horapunta[horapunta['MediaCochesDía_HoraPunta'].notna()].drop_duplicates(subset=['CLinea','FServicio']).assign(IDFranja='HoraPunta',Coches=horapunta['MediaCochesDía_HoraPunta'],Intervalo='070000 - 095959').reset_index(drop=True)]).drop(columns=['Diario']).sort_values(by=['CLinea','FServicio'])

In [56]:
# tmp['MediaCochesMes_Laborables'] = tmp['MediaCochesMes_Laborables'].bfill()

In [ ]:
# # Cuando concateno los dos conjuntos de datos tengo que crear una nueva fila
# # para la IDFranaja => 'HoraPunta' que tenga como número de Coches => 'MediaCochesDía_HoraPunta' 

# oferta_horapunta = pd.concat([\
#        oferta,\
#        horapunta.loc[:,['CLinea', 'Elinea', 'Denominacion', 'FServicio', 'IDFranja','Intervalo', 'Coches', 'Diario', 'fechaCorta', 'MediaCochesMes',  'MediaCochesMes_Laborables']].drop_duplicates(subset=['CLinea','FServicio'])#.sort_values(by=['CLinea','FServicio']).reset_index(drop=True)\
#               .assign(IDFranja='HoraPunta',Coches=horapunta['MediaCochesDía_HoraPunta'],Intervalo='070000 - 095959',MediaCochesMes=horapunta['MediaCochesMes_HoraPunta'],MediaCochesMes_Laborables=horapunta['MediaCochesMes_HoraPuntaLaborables']),]).sort_values(by=['CLinea','FServicio','IDFranja']).reset_index(drop=True)

# oferta_horapunta.loc[(oferta_horapunta['IDFranja']=='HoraPunta'),:] # &(oferta_horapunta['MediaCochesMes_Laborables'].isna())

# sorted(list(set(oferta_horapunta['CLinea'].unique()) - set(oferta_horapunta[oferta_horapunta['IDFranja']=='HoraPunta']['CLinea'].unique())))

In [ ]:
# Sí haces backward y forward en líneas donde no hay franja de HoraPunta se completa con datos que no corresponden.

tmp = oferta[oferta['CLinea']==501].copy()
 
tmp['MediaCochesMes_HoraPunta'] = tmp[(tmp['IDFranja'].isin((*HORAS,'HoraPunta')))].groupby(by=['CLinea','fechaCorta'])['MediaCochesMes'].bfill()

In [ ]:
oferta[(oferta['IDFranja'].isin((*HORAS,'HoraPunta')))]

In [82]:
oferta.to_csv('../datos/output/oferta_diaria_media.csv',encoding='utf8',index=False)
# medias.to_csv('datos/output/medias.csv',encoding='utf8',index=False)
# calendar_dates.drop_duplicates().reset_index(drop=True)\
#                 .to_csv('datos/output/calendario_festivos.csv',encoding='utf8',index=False)

In [81]:
oferta.to_pickle(path='../datos/output/pickles/oferta_diaria_media.pickle')
# medias.to_pickle('../datos/output/medias.pickle')
# calendar_dates.drop_duplicates().reset_index(drop=True)\
#                 .to_pickle('../datos/output/calendario_festivos.pickle')